In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import re
import nltk
import numpy as np

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers import SimpleRNN
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

%matplotlib inline

In [2]:
seed = 8675309

In [3]:
tf.random.set_seed(seed)
np.random.seed(seed)

In [4]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [5]:
train_df.head(2)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


In [6]:
test_df.head(2)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."


In [7]:
train_df.describe(include='all')

,id,keyword,location,text,target
count,7613.000000,7552,5080,7613,7613.00000
unique,NaN,221,3341,7503,NaN
top,NaN,fatalities,USA,11-Year-Old Boy Charged With Manslaughter of T...,NaN
freq,NaN,45,104,10,NaN
mean,5441.934848,NaN,NaN,NaN,0.42966
std,3137.116090,NaN,NaN,NaN,0.49506
min,1.000000,NaN,NaN,NaN,0.00000
25%,2734.000000,NaN,NaN,NaN,0.00000
50%,5408.000000,NaN,NaN,NaN,0.00000
75%,8146.000000,NaN,NaN,NaN,1.00000


In [8]:
train_df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [9]:
# describe yields 10 duplicates but unique does not.
train_df["text"].unique

<bound method Series.unique of 0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
                              ...                        
7608    Two giant cranes holding a bridge collapse int...
7609    @aria_ahrary @TheTawniest The out of control w...
7610    M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611    Police investigating after an e-bike collided ...
7612    The Latest: More Homes Razed by Northern Calif...
Name: text, Length: 7613, dtype: object>

In [10]:
train_df.shape

(7613, 5)

In [11]:
train_df["text"][2]

"All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"

In [38]:
# leaving test set alone for now
dataset = train_df.values

In [39]:
dataset.shape

(7613, 5)

In [40]:
X = train[:, 3]

In [41]:
y = train[:, -1]

In [43]:
y

array([1, 1, 1, ..., 1, 1, 1], dtype=object)

In [44]:
X[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [45]:
y[0]

1

In [46]:
# turns words into numbers
tokenizer = Tokenizer(filters='\t\n', char_level=True)
tokenizer.fit_on_texts(X)

num_words = len(tokenizer.word_index)+1
X = tokenizer.texts_to_sequences(X)

In [49]:
X[0][0]

5

In [48]:
max_log_length = 1024
X_processed = sequence.pad_sequences(X, maxlen=max_log_length)

In [52]:
X_processed[0][0]

0

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.30, random_state=seed)

In [55]:
X_train.shape

(5329, 1024)

In [56]:
X_test.shape

(2284, 1024)

In [57]:
y_train.shape

(5329,)

In [58]:
y_test.shape

(2284,)

In [59]:
X_train[0]

array([ 0,  0,  0, ...,  9, 10, 43], dtype=int32)

In [60]:
X_test[0]

array([ 0,  0,  0, ...,  2, 16, 15], dtype=int32)

In [61]:
y_train[0]

1

In [62]:
y_test[0]

0

# Model 1 : Simple RNN

In [63]:
rnn_model = Sequential()

In [64]:
rnn_model.add(Embedding(num_words, 32, input_length=max_log_length))

In [65]:
rnn_model.add(SimpleRNN(units=32, activation='relu'))

In [66]:
rnn_model.add(Dense(units=1, activation = 'sigmoid'))

In [76]:
rnn_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [77]:
rnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1024, 32)          3072      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 5,185
Trainable params: 5,185
Non-trainable params: 0
_________________________________________________________________


In [78]:
X_train = np.asarray(X_train).astype('float32')

In [79]:
y_train = np.asarray(y_train).astype('float32')

In [80]:
rnn_model.fit(x=X_train, y=y_train, epochs=3, batch_size = 128, validation_split=0.25)

Epoch 1/3
32/32 [==============================] - 11s 337ms/step - loss: 0.6891 - accuracy: 0.5435 - val_loss: 0.6813 - val_accuracy: 0.5641
Epoch 2/3
32/32 [==============================] - 11s 335ms/step - loss: 0.6723 - accuracy: 0.5678 - val_loss: 0.6685 - val_accuracy: 0.5671
Epoch 3/3
32/32 [==============================] - 9s 285ms/step - loss: 0.6562 - accuracy: 0.5888 - val_loss: 0.6616 - val_accuracy: 0.5806


In [82]:
X_test = np.asarray(X_test).astype('float32')

In [83]:
y_test = np.asarray(y_test).astype('float32')

In [84]:
rnn_model.evaluate(x=X_test, y=y_test, batch_size = 128)

18/18 [==============================] - 1s 71ms/step - loss: 0.6521 - accuracy: 0.5976


[0.6520897746086121, 0.5976357460021973]

# Model 2 : LSTM + Dropout Layers

In [85]:
lstm_model = Sequential()
lstm_model.add(Embedding(num_words, 32, input_length=max_log_length))
lstm_model.add(LSTM(units=64, recurrent_dropout=0.5))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(units=1, activation = 'relu'))

In [86]:
lstm_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [87]:
lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1024, 32)          3072      
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 27,969
Trainable params: 27,969
Non-trainable params: 0
_________________________________________________________________


In [88]:
lstm_model.fit(x=X_train, y=y_train, epochs=3, batch_size = 128, validation_split=0.25)

Epoch 1/3
32/32 [==============================] - 52s 2s/step - loss: 0.9932 - accuracy: 0.5663 - val_loss: 0.6822 - val_accuracy: 0.5641
Epoch 2/3
32/32 [==============================] - 55s 2s/step - loss: 0.6955 - accuracy: 0.5460 - val_loss: 0.6702 - val_accuracy: 0.5611
Epoch 3/3
32/32 [==============================] - 57s 2s/step - loss: 0.6746 - accuracy: 0.5848 - val_loss: 0.6657 - val_accuracy: 0.5716


In [89]:
lstm_model.evaluate(x=X_test, y=y_test, batch_size = 128)

18/18 [==============================] - 5s 274ms/step - loss: 0.6499 - accuracy: 0.5849


[0.6499080061912537, 0.5849387049674988]

# Model 3 : LSTM + Dropout Layers

In [90]:
my_model = Sequential()
my_model.add(Embedding(num_words, 32, input_length=max_log_length))
my_model.add(Dense(units=32, activation = 'relu'))
my_model.add(LSTM(units=64, recurrent_dropout=0.5))
my_model.add(Dropout(0.5))
my_model.add(Dense(units=1, activation = 'relu'))

In [91]:
my_model.compile(loss = 'binary_crossentropy', optimizer = 'sgd', metrics = ['accuracy', 'mae'])

In [92]:
my_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1024, 32)          3072      
_________________________________________________________________
dense_3 (Dense)              (None, 1024, 32)          1056      
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 29,025
Trainable params: 29,025
Non-trainable params: 0
_________________________________________________________________


In [93]:
my_model.fit(x=X_train, y=y_train, epochs=3, batch_size = 128, validation_split=0.25)

Epoch 1/3
32/32 [==============================] - 72s 2s/step - loss: 8.4215 - accuracy: 0.4387 - mae: 1.2402 - val_loss: 8.6027 - val_accuracy: 0.4359 - val_mae: 1.2738
Epoch 2/3
32/32 [==============================] - 60s 2s/step - loss: 8.6555 - accuracy: 0.4322 - mae: 1.2612 - val_loss: 8.6027 - val_accuracy: 0.4359 - val_mae: 1.2725
Epoch 3/3
32/32 [==============================] - 59s 2s/step - loss: 8.6554 - accuracy: 0.4322 - mae: 1.2724 - val_loss: 8.6027 - val_accuracy: 0.4359 - val_mae: 1.2718


In [94]:
my_model.evaluate(x=X_test, y=y_test, batch_size = 128)

18/18 [==============================] - 6s 308ms/step - loss: 8.8197 - accuracy: 0.4216 - mae: 1.2858


[8.819721221923828, 0.4216287136077881, 1.2858084440231323]

# Submitting Best Model

In [ ]:
to_csv("submission.csv", index=False)